In [1]:
running = False

## Physics Model
First, we define a constants class that stores all of the physical constants that the BSS model uses (following AdEx). The "default" object is AdEx, but there are also a couple of simple objects representing the Plateau Potential model (for dendrites) and the LIF model.

In [3]:
from copy import copy

class PhysicalProperties:
    def __init__(self,
        C_m, # membrane capacitance (F). You can think of this as a conversion between current and changes in voltage.
        g_leak, # leak conductance (S). Controls how fast the neuron likes to fall back to its leak potential.
        V_leak, # leak potential (V). The leak (default) potential of the neuron when it isn't stimulated with current.
        delta_exp, # exponential slope factor (V). Controls how quickly the neuron spikes once it reaches the exponential threshold.
        V_exp, # exponential threshold (V). Once potential reaches this threshold, it will spike very quickly.
        T_w, # adaptation time constant (s). Controls how quickly the neuron adapts to itself.
        a, # adaptation scaling factor (S). Controls how strongly the neuron adapts to itself.
        b, # spike adaptation amount (A). Controls how much the adaptation increases by for each output spike of the neuron.
        V_r, # reset potential (V). After a spike, the neuron is forced to this potential using g_reset for a period of time t_r.
        V_th, # hard firing threshold (V). This determines at what potential the neuron actually fires.
        t_r, # refractory period (s). The duration for which the neuron is forced to V_r after firing.
        T_syn, # synaptic time constant (s). This controls how quickly synapse strength decays over time.
        g_reset # reset conductance (S). Controls how fast the neuron jumps to reset potential after firing.
    ):
        self.C_m = C_m
        self.g_leak = g_leak
        self.V_leak = V_leak
        self.delta_exp = delta_exp
        self.V_exp = V_exp
        self.T_w = T_w
        self.a = a
        self.b = b
        self.V_r = V_r
        self.V_th = V_th
        self.t_r = t_r
        self.T_syn = T_syn
        self.g_reset = g_reset

AdEx = PhysicalProperties( # I found these in the 2005 AdEx paper, and I made up t_r, T_syn, and g_reset since they weren't specified.
        C_m = 281e-12,
        g_leak = 30e-9,
        V_leak = -70.6e-3,
        delta_exp = 2e-3,
        V_exp = -50.4e-3,
        T_w = 144e-3,
        a = 4e-9,
        b = 0.0805e-9,
        V_r = -80.6e-3,
        V_th = 20e-3,
        t_r = 0.04,
        T_syn = 1,
        g_reset = 1e-7
)

Plateau_Potential = copy(AdEx)
Plateau_Potential.V_th = 0 # This one fires instead of going exponential
Plateau_Potential.V_r = 200e-3 # When it fires, potential is actually forced above the threshold before falling back down (plateau)
Plateau_Potential.t_r = 0.2 # Dendrite plateaus typically last for a while
Plateau_Potential.V_exp = 1000 # These never go exponential, so this is an arbitrary large number

LIF = copy(AdEx)
LIF.V_th = -50.4e-3 # This also fires instead of going exponential
LIF.V_exp = 1000 # Like Plateau_Potential, these never go exponential

Passive = copy(AdEx)
Passive.V_exp = 1000 # also never go exponential
Passive.a = 0 # no adaptation 
Passive.b = 0 # no adaptation
Passive.V_th = 1000

## First Step: Python Class
I started with a non-Lava python class to emulate the BSS neuron behavior. It simulates only one neuron (no synapses or components) and doesn't account for things like reset conductance and situations where V_r > V_th.

In [4]:
import numpy as np

class Neuron:
    def __init__(self, physical_properties):
        self.pp = physical_properties
        self.w = 0 # adaptation current
        self.V_m = self.pp.V_leak # membrane voltage
        self.I = 0 # input current
        self.fired_timestamp = None # last time of fire
    
    def I_syn(self, t): # current from synapses (not implemented in this class, see Lava class)
        return 0
    
    def I_stim(self, t): # current from other components (also not implemented in this class, see Lava class)
        return 0
    
    def dV_m_dt(self, t): # get the derivative of potential at a given time, depending on current and potential
        exponential_component = np.exp((self.V_m - self.pp.V_exp)/self.pp.delta_exp)
        return (self.pp.g_leak * (self.pp.V_leak - self.V_m) + 
                self.pp.g_leak * self.pp.delta_exp * exponential_component + 
                self.I - self.w) / self.pp.C_m
    
    def dw_dt(self, t): # get the derivative of the adaptation current at a given time, depending on w and potential
        return (self.pp.a * (self.V_m - self.pp.V_leak) - self.w) / self.pp.T_w
    
    def step(self, t, dt): # take a step of length dt and change the neuron's properties accordingly
        self.I = self.I_syn(t) + self.I_stim(t)
        dw = self.dw_dt(t) * dt
        dV_m = self.dV_m_dt(t) * dt
        self.w += dw
        
        if self.fired_timestamp is not None:
            self.V_m = self.pp.V_r
            if t - self.fired_timestamp >= self.pp.t_r:
                self.fired_timestamp = None
        else:
            self.V_m += dV_m
            if(self.V_m > self.pp.V_th):
                self.V_m = self.pp.V_th
                self.w += self.pp.b
                self.fired_timestamp = t
        
        

## User Input
Here is a simple extension of that python class which takes a lambda function as input and applies that as a current over time.

In [5]:
class ManualInputNeuron(Neuron): # fn is a lambda from time (seconds) to current (A)
    def __init__(self, physical_properties, fn):
        super().__init__(physical_properties)
        self.fn = fn
    def I_syn(self, t):
        return self.fn(t)

## Graphing
Here we define a view functions to graph voltages, currents, and spike times (plot_spikes was copied from a Lava tutorial). Also, a function to test out the python class above.

In [6]:
import matplotlib.pyplot as plt

def plot_voltage(name, times, V_ms, legend = None, min_axis = None, max_axis = None):
    plt.plot(times, V_ms)
    plt.xlabel('Time (s)')  
    plt.ylabel('Membrane Voltage (V)')  
    plt.ylim(min_axis, max_axis)
    plt.title(f'{name} - Voltage')
    if legend is not None:
        plt.legend(legend)
    plt.show()  

def plot_current(name, times, Is, legend = None):
    plt.plot(times, Is)
    plt.xlabel('Time (s)')  
    plt.ylabel('Current (A)')
    plt.title(f'{name} - Current')
    if legend is not None:
        plt.legend(legend)
    plt.show()

# Given a graph name, time data, voltage data, and current data, make graphs of V and I over time.
def plot_voltage_and_current(name, times, V_ms, Is, legend = None, min_voltage_axis = None, max_voltage_axis = None):
    plot_voltage(name, times, V_ms, legend, min_voltage_axis, max_voltage_axis)
    plot_current(name, times, Is, legend)
    

# Graph spike data over time (can accept multiple spike trains)
def plot_spikes(spikes, legend, colors):
    offsets = list(range(1, len(spikes) + 1))
    
    plt.figure(figsize=(10, 3))
    
    spikes_plot = plt.eventplot(positions=spikes, 
                                lineoffsets=offsets,
                                linelength=0.9,
                                colors=colors)
    
    plt.title("Spike arrival")
    plt.xlabel("Milliseconds")
    plt.ylabel("Neurons")
    plt.yticks(ticks=offsets, labels=legend)
    
    plt.show()

# Test the Neuron class above for a certain amount of time and plot the results
def test_neuron(test_name, neuron, timestep, total_time, min_voltage_axis = None, max_voltage_axis = None):
    time = 0
    V_ms = []
    Is = []
    times = []
    while time <= total_time:
        neuron.step(time, timestep)
        V_ms.append(neuron.V_m)
        times.append(time)
        Is.append(neuron.I)
        time += timestep
    plot_voltage_and_current(test_name, times, V_ms, Is, min_voltage_axis, max_voltage_axis)

## Testing the Python Class
Now we can test the python class, using a standard AdEx neuron and a step function for current input.

In [2]:
# Here, we are testing an AdEx neuron that receives a step function of manual current input (see current graph below).
# The test runs for a total duration of 1s at 1ms intervals.
if running:
    test_neuron("Test 1", ManualInputNeuron(AdEx, lambda t: 0.5e-9 if t < 0.2 else 0.8e-9 if t > 0.5 else 0), 0.001, 1)

## Lava Process
Here is the BSS Neuron Process for Lava. As you can see, it taks many variables, since Lava doesn't have the concept of structs, objects, or references. The first section of variables corresponds to physical constants, the second section corresponds to weights/conductances, the third section corresponds to state variables like time, current, and voltage, and the fourth section has all of the IO ports.

In [8]:
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import InPort, OutPort, RefPort
import numpy as np
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.magma.core.model.py.ports import PyInPort, PyOutPort, PyRefPort
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.resources import CPU
from lava.magma.core.decorator import implements, requires, tag
from lava.magma.core.model.py.model import PyLoihiProcessModel
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps

# This class represents an arbitrary number of neurons, but they all share the same physical properties.
# For example, it could represent a layer of soma or a layer of a specific type of dendrite.
class BSSNeuronProcess(AbstractProcess):
    def __init__(self, dt, shape = (1,), name = 'BSSNeuronProcess',
        physical_properties = AdEx
    ):
        super().__init__(name=name)
        # See abstract model class below for documentation on properties
        self.C_m = Var(shape=(1,), init=physical_properties.C_m)
        self.g_leak = Var(shape=(1,), init=physical_properties.g_leak)
        self.V_leak = Var(shape=(1,), init=physical_properties.V_leak)
        self.delta_exp = Var(shape=(1,), init=physical_properties.delta_exp)
        self.V_exp = Var(shape=(1,), init=physical_properties.V_exp)
        self.T_w = Var(shape=(1,), init=physical_properties.T_w)
        self.a = Var(shape=(1,), init=physical_properties.a)
        self.b = Var(shape=(1,), init=physical_properties.b)
        self.V_r = Var(shape=(1,), init=physical_properties.V_r)
        self.V_th = Var(shape=(1,), init=physical_properties.V_th)
        self.t_r = Var(shape=(1,), init=physical_properties.t_r)
        self.T_syn = Var(shape=(1,), init=physical_properties.T_syn)
        self.g_reset = Var(shape=(1,), init=physical_properties.g_reset)
        
        self.g_above = Var(shape=shape, init=0)
        self.g_left = Var(shape=shape, init=0)
        self.g_right = Var(shape=shape, init=0)
        self.g_below = Var(shape=shape, init=0)
        
        self.dt = Var(shape=(1,), init=dt)
        self.I = Var(shape=shape, init=0)
        self.w = Var(shape=shape, init=0)
        self.V_m = Var(shape=shape, init=physical_properties.V_leak)
        self.t = Var(shape=(1,), init=0)
        self.fired_timestamp = Var(shape=shape, init=-1)
        
        self.spike_out = OutPort(shape=shape)
        self.synapse_in = InPort(shape=shape)
        self.V_m_out = OutPort(shape=shape)
        self.above_in = InPort(shape=shape)
        self.left_in = InPort(shape=shape)
        self.right_in = InPort(shape=shape)
        self.below_in = InPort(shape=shape)
    def update_physical_properties(self, physical_properties):
        self.C_m.init = physical_properties.C_m
        self.g_leak.init = physical_properties.g_leak
        self.V_leak.init = physical_properties.V_leak
        self.delta_exp.init = physical_properties.delta_exp
        self.V_exp.init = physical_properties.V_exp
        self.T_w.init = physical_properties.T_w
        self.a.init = physical_properties.a
        self.b.init = physical_properties.b
        self.V_r.init = physical_properties.V_r
        self.V_th.init = physical_properties.V_th
        self.t_r.init = physical_properties.t_r
        self.T_syn.init = physical_properties.T_syn
        self.g_reset.init = physical_properties.g_reset
        self.V_m.init = physical_properties.V_leak

from enum import Enum
class Direction(Enum):
    ABOVE = 0
    LEFT = 1
    RIGHT = 2
    BELOW = 3

def get_corresponding_in_port(neuron, direction):
    if direction == Direction.ABOVE:
        return neuron.above_in
    if direction == Direction.LEFT:
        return neuron.left_in
    if direction == Direction.RIGHT:
        return neuron.right_in
    if direction == Direction.BELOW:
        return neuron.below_in
    assert False, "unexpected direction."

def get_corresponding_conductance(neuron, direction):
    if direction == Direction.ABOVE:
        return neuron.g_above
    if direction == Direction.LEFT:
        return neuron.g_left
    if direction == Direction.RIGHT:
        return neuron.g_right
    if direction == Direction.BELOW:
        return neuron.g_below
    assert False, "unexpected direction."

def get_in_direction(out_direction):
    if out_direction == Direction.ABOVE:
        return Direction.BELOW
    if out_direction == Direction.LEFT:
        return Direction.RIGHT
    if out_direction == Direction.RIGHT:
        return Direction.LEFT
    if out_direction == Direction.BELOW:
        return Direction.ABOVE

def connect_neurons(first, second, out_direction, g):
    first_in_port = get_corresponding_in_port(first, out_direction)
    second_in_port = get_corresponding_in_port(second, get_in_direction(out_direction))
    assert (not first_in_port.in_connections) and (not second_in_port.in_connections), "local neuron ports can only have one connection!"
    
    first.V_m_out.connect(get_corresponding_in_port(second, get_in_direction(out_direction)))
    second.V_m_out.connect(get_corresponding_in_port(first, out_direction))

    get_corresponding_conductance(first, out_direction).init = g
    get_corresponding_conductance(second, get_in_direction(out_direction)).init = g

## Lava Model
Here is the model implementation of the process above. I made it "abstract" (i.e. not technically an implementation) so that it can serve as a parent class to more specific implementations. As you can see, it stores the same variables as the process (required by Lava). See function documentation for more details.

In [9]:
class AbstractBSSNeuronProcessModel(PyLoihiProcessModel):
    C_m: float = LavaPyType(float, float) # These are the same physical properties as in the PhysicalProperties class above
    g_leak: float = LavaPyType(float, float)
    V_leak: float = LavaPyType(float, float)
    delta_exp: float = LavaPyType(float, float)
    V_exp: float = LavaPyType(float, float)
    T_w: float = LavaPyType(float, float)
    a: float = LavaPyType(float, float)
    b: float = LavaPyType(float, float)
    V_r: float = LavaPyType(float, float)
    V_th: float = LavaPyType(float, float)
    t_r: float = LavaPyType(float, float)
    T_syn: float = LavaPyType(float, float)
    g_reset: float = LavaPyType(float, float)
    
    g_above: np.ndarray = LavaPyType(np.ndarray, float) # The conductance (weight) for this neuron's optional connection to a compartment above
    g_left: np.ndarray = LavaPyType(np.ndarray, float) # The conductance (weight) for this neuron's optional connection to a compartment on the left
    g_right: np.ndarray = LavaPyType(np.ndarray, float) # The conductance (weight) for this neuron's optional connection to a compartment on the right
    g_below: np.ndarray = LavaPyType(np.ndarray, float) # The conductance (weight) for this neuron's optional connection to a compartment below
    
    dt: float = LavaPyType(float, float) # The time interval for this neuron to run at
    I: np.ndarray = LavaPyType(np.ndarray, float) # The sum of manual currents + compartment currents + synapse currents, used for logging
    w: np.ndarray = LavaPyType(np.ndarray, float) # The adaptation currents of this neuron layer
    V_m: np.ndarray = LavaPyType(np.ndarray, float) # The membrane voltages of this neuron layer
    t: float = LavaPyType(float, float) # The current time
    fired_timestamp: np.ndarray = LavaPyType(np.ndarray, float) # A list of timestamps for when each neuron in the layer last fired
    
    spike_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, bool, precision=1) # Send a list of bools where each True represents a fire at that index
    synapse_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, float) # Receives a list of the form sent by synapse_out
    V_m_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, float) # sends all potentials from this neuron layer
    above_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, float) # Connects to a V_m_out of another neuron layer
    left_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, float) # Connects to a V_m_out of another neuron layer
    right_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, float) # Connects to a V_m_out of another neuron layer
    below_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, float) # Connects to a V_m_out of another neuron layer

    # I_MC is updated based on the difference between the membrane voltage of each connection and this neuron's voltage
    def I_MC(self, t):
        total = np.zeros(np.shape(self.I))
        if t == 0:
            return total # can't expect anything on first step
        for port, conductance in ((self.above_in, self.g_above), 
                                  (self.left_in, self.g_left), 
                                  (self.right_in, self.g_right), 
                                  (self.below_in, self.g_below)):
            if not port.csp_ports:
                continue # if port isn't connected
            total += np.multiply(conductance, (port.recv() - self.V_m))
        return total
        
    # Get the currents in this layer based on synapses, using the synapse weight matrix
    # This current deteriorates exponentialfly over time (given in BSS paper), don't really understand why
    def I_syn(self, t):
        if not self.synapse_in.csp_ports or t == 0:
            return np.zeros(np.shape(self.I)) # if neuron doesn't receive from synapse, also can't expect anything on first step
        weighted_in = self.synapse_in.recv()
        return weighted_in * np.exp(-t / self.T_syn)

    # Get the current from user input (overridden in child classes)
    def I_stim(self, t):
        return np.zeros(np.shape(self.I))

    # Get the derivative of membrane voltage, based on current and voltage
    def dV_m_dt(self):
        exponential_component = np.exp((self.V_m - self.V_exp)/self.delta_exp)
        return (self.g_leak * (self.V_leak - self.V_m) + 
                self.g_leak * self.delta_exp * exponential_component + 
                self.I - self.w) / self.C_m

    # Get the derivative of the adaptation current, based on w and voltage
    def dw_dt(self):
        return (self.a * (self.V_m - self.V_leak) - self.w) / self.T_w

    # Update the neuron layer
    def run_spk(self):
        which_fired = self.fired_timestamp >= 0 # Check which neurons recently fired
        which_are_done_resting = np.logical_and(which_fired, (self.t - self.fired_timestamp >= self.t_r)) # Check which of those should stop resting
        self.fired_timestamp[which_are_done_resting] = -1 # Tell those ones to stop resting

        
        self.I = self.I_syn(self.t) + self.I_MC(self.t) + self.I_stim(self.t) # Calculate current 
        dw_dt = self.dw_dt() # Get change in adaptation current
        dV_m_dt = np.where(which_fired, (self.g_reset * (self.V_r - self.V_m))/self.C_m, self.dV_m_dt()) # Get change in potential
        # ^ Note that the neurons in the layer which fired recently will behave differently, clamping to V_r instead of being affected by dV_m_dt
        
        self.w += dw_dt * self.dt # Change adaptation current based on derivative
        which_below_threshold = self.V_m < self.V_th # Check which neurons in the layer are below firing threshold
        self.V_m += dV_m_dt * self.dt # Update potential
        which_above_threshold = self.V_m >= self.V_th # Check which neurons in the layer are above firing threshold

        
        which_should_fire = np.logical_and(which_below_threshold, which_above_threshold) # If a neuron transitioned from below to above, it should fire
        self.w[which_should_fire] += self.b # Increase adaptation currents for firing neurons
        self.V_m[which_should_fire] = self.V_th # Clamp firing neuron potentials to V_th so they don't get out of control
        self.fired_timestamp[which_should_fire] = self.t # Update timestamps
        self.spike_out.send(which_should_fire) # Send spike signal
        self.V_m_out.send(self.V_m)
        self.t += self.dt # Increment time counter

## Child Lava Models
Here are some more child classes building off of the process/model written above. The first model is exactly the same as the abstract process model, except that it's no longer abstract. The next process/model pair written here can be used to feed user input as current into neurons. Unfortunately Lava has no concept of lambda functions, so if you want behavior that's not a simple step function, you'll have to directly edit this model or make a new one.

In [10]:
# An exact copy of AbstractBSSNeuronProcessModel
@implements(proc=BSSNeuronProcess, protocol=LoihiProtocol)
@requires(CPU)
class BSSNeuronProcessModel(AbstractBSSNeuronProcessModel):
    pass

# This class accepts a step function as input (can vary per neuron) and will apply that to the layer
class BSSNeuronProcessManualInput(BSSNeuronProcess):
    def __init__(self, dt, shape = (1,), name = 'BSSNeuronProcessManualInput', physical_properties = AdEx,
                 times = (), heights = ()):
        super().__init__(dt=dt, shape=shape, name=name, physical_properties=physical_properties)
        self.heights = Var(shape = np.shape(heights), init=heights)
        self.times = Var(shape= np.shape(times), init=times)

# A model implementing the process above
@implements(proc=BSSNeuronProcessManualInput, protocol=LoihiProtocol)
@requires(CPU)
class BSSNeuronProcessModelManualInput(AbstractBSSNeuronProcessModel):
    times: np.ndarray = LavaPyType(np.ndarray, float)
    heights: np.ndarray = LavaPyType(np.ndarray, float)
    
    def I_stim(self, t):
        heights = np.zeros(np.shape(self.I))
        for i, time in enumerate(self.times):
            heights = np.where(t >= time, self.heights[i], heights)
        return heights

# This class accepts a step function as input (can vary per neuron) and will apply that to the layer
class BSSNeuronProcessTriangleSpike(BSSNeuronProcess):
    def __init__(self, dt, shape = (1,), name = 'BSSNeuronProcessManualInput', physical_properties = AdEx,
                 starts = 0, heights = 0, uptimes = 1, downtimes = 1):
        super().__init__(dt=dt, shape=shape, name=name, physical_properties=physical_properties)
        self.starts = Var(shape = shape, init=starts)
        self.heights = Var(shape = shape, init=heights)
        self.uptimes = Var(shape = shape, init=uptimes)
        self.downtimes = Var(shape = shape, init=downtimes)

# A model implementing the process above
@implements(proc=BSSNeuronProcessTriangleSpike, protocol=LoihiProtocol)
@requires(CPU)
class BSSNeuronProcessModelTriangleSpike(AbstractBSSNeuronProcessModel):
    starts: np.ndarray = LavaPyType(np.ndarray, float)
    heights: np.ndarray = LavaPyType(np.ndarray, float)
    uptimes: np.ndarray = LavaPyType(np.ndarray, float)
    downtimes: np.ndarray = LavaPyType(np.ndarray, float)
    
    def I_stim(self, t):
        t_heights = np.zeros(np.shape(self.I))
        
        which_going_up = np.logical_and(t > self.starts, t <= self.starts + self.uptimes)
        t_heights = np.where(which_going_up, np.multiply(self.heights, np.divide(t - self.starts, self.uptimes)), t_heights)
        
        which_going_down = np.logical_and(t > self.starts + self.uptimes, t <= self.starts + self.uptimes + self.downtimes)
        t_heights = np.where(which_going_down, np.multiply(self.heights, 1 - np.divide(t - (self.starts + self.uptimes), self.downtimes)), t_heights)
        
        return t_heights

## Synapse processes and models

In [11]:
class BSSSynapseProcess(AbstractProcess):
    def __init__(self, in_shape = (1,), out_shape=(1,), name = 'BSSSynapseProcess',
        synapse_weights = 0
    ):
        super().__init__(name=name)
        self.synapse_weights = Var(shape=out_shape+in_shape, init=synapse_weights)
        
        self.synapse_in = InPort(shape=in_shape)
        self.synapse_out = OutPort(shape=out_shape)

class AbstractBSSSynapseProcessModel(PyLoihiProcessModel):
    synapse_weights: np.ndarray = LavaPyType(np.ndarray, float) # A weight matrix for synapses, used to connect this layer of neurons
                                                                # to another layer of the same shape (or it can feed into itself, but this would be rare)
    synapse_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, bool, precision=1)
    synapse_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, float)

    def run_spk(self):
        if self.synapse_in.csp_ports and self.synapse_out.csp_ports:
            in_results = self.synapse_in.recv()
            self.synapse_out.send(self.synapse_weights @ in_results)

@implements(proc=BSSSynapseProcess, protocol=LoihiProtocol)
@requires(CPU)
class BSSSynapseProcessModel(AbstractBSSSynapseProcessModel):
    pass
            

## Monitoring Classes
These classes just make it a little easier to monitor process data using Lava Monitor objects.

In [12]:
from lava.proc.monitor.process import Monitor

# Wraps a Monitor object
class NeuronMonitor:
    def __init__(self, neuron_name, var, var_name, num_steps):
        self.monitor = Monitor()
        self.monitor.probe(var, num_steps)
        self.neuron_name = neuron_name
        self.var_name = var_name
    def get_neuron_data(self):
        return self.monitor.get_data()[self.neuron_name][self.var_name]

# Wraps a few NeuronMonitor objects that are commonly used
class StandardNeuronMonitors:
    def __init__(self, neuron, num_steps):
        self.V_m_monitor = NeuronMonitor(neuron.name, neuron.V_m, 'V_m', num_steps)
        self.I_monitor = NeuronMonitor(neuron.name, neuron.I, 'I', num_steps)
        self.spike_monitor = NeuronMonitor(neuron.name, neuron.spike_out, 'spike_out', num_steps)

## AdEx Reacting to Step Function
This experiment is designed to imitate Figure 2C in 'Adaptive Exponential Integrate-and-Fire Model as an Effective Description of Neuronal Activity', 2005, by Brette and Gerstner. 

In [3]:
if running:
    total_time = 1
    dt = 0.001
    dendrites_name = 'DendriteProcess'
    num_neurons = 1
    shape = (num_neurons,)
    model_tag = None
    
    num_steps = (int)(total_time // dt)
    
    dendrites = BSSNeuronProcessManualInput(name=dendrites_name, dt=dt, shape=shape,
                                # since no physics was specified, this will be an AdEx neuron layer
                                heights = (
                                    0.5e-9, 0, 0.8e-9 # step function as shown in the paper
                                ), times = (
                                    0, 0.2, 0.5
                                ))
    
    dendrites_monitors = StandardNeuronMonitors(dendrites, num_steps)
    
    run_cfg = Loihi1SimCfg(select_tag=model_tag)
    dendrites.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_cfg)
    
    plot_voltage_and_current('AdEx Neuron 1', [i * dt for i in range(num_steps)], 
                             dendrites_monitors.V_m_monitor.get_neuron_data(), 
                             dendrites_monitors.I_monitor.get_neuron_data())
    
    dendrites.stop()

## AdEx Reacting to Current Spike
This experiment is meant to imitate Figure 2A in 'Emulating Dendritic Computing Paradigms On Analog Neuromorphic Hardware', 2022, by Kaiser et al (the paper that describes the BSS-2). 

In [4]:
if running:
    total_time = 0.25
    dt = 0.001
    dendrites_name = 'DendriteProcess'
    num_neurons = 1
    shape = (num_neurons,)
    model_tag = None
    
    num_steps = (int)(total_time // dt)
    
    dendrites = BSSNeuronProcessManualInput(name=dendrites_name, dt=dt, shape=shape,
                                physical_properties = AdEx, # this neuron explicitly tells us that it's an AdEx layer
                                heights = (
                                    0.8e-9, 0 # this step function represents a very short current spike
                                ), times = (
                                    0.01, 0.03
                                ))
    
    dendrites_monitors = StandardNeuronMonitors(dendrites, num_steps)
    
    run_cfg = Loihi1SimCfg(select_tag=model_tag)
    dendrites.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_cfg)
    
    plot_voltage_and_current('AdEx Neuron 2', [i * dt for i in range(num_steps)], 
                             dendrites_monitors.V_m_monitor.get_neuron_data(), 
                             dendrites_monitors.I_monitor.get_neuron_data())
    
    dendrites.stop()

## LIF Reacting to Current Spike
This experiment is meant to imitate Figure 2B in 'Emulating Dendritic Computing Paradigms On Analog Neuromorphic Hardware', 2022, by Kaiser et al (the paper that describes the BSS-2). 

In [5]:
if running:
    total_time = 0.25
    dt = 0.001
    dendrites_name = 'DendriteProcess'
    num_neurons = 1
    shape = (num_neurons,)
    model_tag = None
    
    num_steps = (int)(total_time // dt)
    
    dendrites = BSSNeuronProcessManualInput(name=dendrites_name, dt=dt, shape=shape,
                                physical_properties = LIF, # note this important different - this is now a LIF neuron layer
                                heights = (
                                    0.8e-9, 0 # the same step function as above, a short current spike
                                ), times = (
                                    0.01, 0.03
                                ))
    
    dendrites_monitors = StandardNeuronMonitors(dendrites, num_steps)
    
    run_cfg = Loihi1SimCfg(select_tag=model_tag)
    dendrites.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_cfg)
    
    plot_voltage_and_current('LIF Neuron', [i * dt for i in range(num_steps)], 
                             dendrites_monitors.V_m_monitor.get_neuron_data(), 
                             dendrites_monitors.I_monitor.get_neuron_data())
    
    dendrites.stop()

## Plateau Potential Reacting to Current Spike
This experiment is meant to imitate Figure 2C in 'Emulating Dendritic Computing Paradigms On Analog Neuromorphic Hardware', 2022, by Kaiser et al (the paper that describes the BSS-2). 

In [6]:
if running:
    total_time = 0.25
    dt = 0.001
    dendrites_name = 'DendriteProcess'
    num_neurons = 1
    shape = (num_neurons,)
    model_tag = None
    
    num_steps = (int)(total_time // dt)
    
    dendrites = BSSNeuronProcessManualInput(name=dendrites_name, dt=dt, shape=shape,
                                physical_properties = Plateau_Potential, # this is a Plateau Potential neuron layer
                                heights = (
                                    2.5e-9, 0 # same step function as before
                                ), times = (
                                    0.01, 0.03
                                ))
    
    dendrites_monitors = StandardNeuronMonitors(dendrites, num_steps)
    
    run_cfg = Loihi1SimCfg(select_tag=model_tag)
    dendrites.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_cfg)
    
    plot_voltage_and_current('Plateau Potential Neuron', [i * dt for i in range(num_steps)], 
                             dendrites_monitors.V_m_monitor.get_neuron_data(), 
                             dendrites_monitors.I_monitor.get_neuron_data())
    
    dendrites.stop()

## Synapses and Compartments
Here is an experiment with two neurons, each with two compartments. The series of events is as follows:
1) A short spike of current is injected into neuron 1 compartment 1, a Plateau Potential dendrite
2) This produces a voltage plateau potential in the dendrite
3) The second compartment, an AdEx soma, registers the dendrite as a connected compartment, so it reads off the dendrite's voltage, producing a current in the soma
4) This produces a voltage in the soma, causing the neuron to spike
5) Neuron 2 compartment 1, another Plateau Potential dendrite, is connected to neuron 1 compartment 2 (think of it as an axon) via synapse, so it receives current from the spike
6) This produces a voltage plateau potential in the second dendrite
7) The second compartment of neuron 2 is connected to the first compartment, so it reads off the voltage and produces a current in the soma of neuron 2
8) This produces a voltage in the soma of neuron 2, causing the neuron to spike
   
Overall, we go from a manual current input on a dendrite on one neuron to a spike from the axon of another neuron. Actually, since the plateau potentials last for so long, the initial short spike of current turns into three output spikes per neuron, as you'll see in the graphs. The blue curves correspond to the first neuron, and the orange curves correspond to the second neuron.

In [7]:
if running:
    total_time = 1
    dt = 0.001
    dendrites_name = 'DendriteProcess'
    somas_name = 'SomaProcess'
    num_neurons = 2 # note that we now have 2 neurons per layer!
    shape = (num_neurons,)
    model_tag = None
    
    num_steps = (int)(total_time // dt)
    
    dendrites = BSSNeuronProcessManualInput(name=dendrites_name, dt=dt, shape=shape,
                                physical_properties=Plateau_Potential,
                                heights = (
                                    (2e-8, 0), # this step function represents a short spike in current at t = 0.3
                                    (0, 0)
                                ), times = (
                                    0.3, 0.32
                                ))
    somas = BSSNeuronProcess(name=somas_name, dt=dt, shape=shape)
    synapse = BSSSynapseProcess(in_shape=shape, out_shape=shape, synapse_weights = (
                                    (0, 0), # this weight matrix specifies that the second dendrite will weigh its first synapse connection
                                    (1e-8, 0) # all other weights are 0
                                ))
    
    connect_neurons(dendrites, somas, Direction.BELOW, 8e-8)
    somas.spike_out.connect(synapse.synapse_in) # connect all (2) somas to all (2) dendrites, forming a 2x2 synapse matrix
    synapse.synapse_out.connect(dendrites.synapse_in)
    
    dendrites_monitors = StandardNeuronMonitors(dendrites, num_steps) # we'll monitor both types of neurons
    somas_monitors = StandardNeuronMonitors(somas, num_steps)
    
    run_cfg = Loihi1SimCfg(select_tag=model_tag)
    somas.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_cfg)
    
    plot_voltage_and_current('dendrites', [i * dt for i in range(num_steps)], 
                             dendrites_monitors.V_m_monitor.get_neuron_data(), 
                             dendrites_monitors.I_monitor.get_neuron_data(), min_voltage_axis = -80e-3, max_voltage_axis=20e-3)
    plot_voltage_and_current('somas', [i * dt for i in range(num_steps)], 
                             somas_monitors.V_m_monitor.get_neuron_data(), 
                             somas_monitors.I_monitor.get_neuron_data(), min_voltage_axis = -80e-3, max_voltage_axis=20e-3)
    
    plot_spikes(spikes=[np.where(somas_monitors.spike_monitor.get_neuron_data()[:, i])[0] for i in range(num_neurons)], 
                legend=[f'{i + 1}' for i in range(num_neurons)], 
                colors=['#ff0000' for _ in range(num_neurons)])
    
    somas.stop()